# 3 
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате. 

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён. 

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

$H_0\colon$ зевота не заразительна, т.е. p1=p2

$H_1\colon$ заразительна, т.е. p1>p2

In [85]:
import numpy as np
from scipy.stats import norm

In [88]:
p1=10/34
p2=4/16
n1=34
n2=16
P=float(p1*n1 + p2*n2) / (n1 + n2)
z=(p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
z

0.32410186177608225

In [89]:
p_value=(1-norm.cdf(z,0,1))
round(p_value,4)

0.3729

=> гипотеза $H_0\colon$ не отвергается

 # 4
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. На рисунке ниже показаны измеренные признаки:

  Загрузите данные:  
banknotes.txt

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

    логистическая регрессия по признакам X1,X2,X3
    логистическая регрессия по признакам X4,X5,X6

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−85.5\times10^{-8}5.5×10−8, нужно ввести 8).
1 балл

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [58]:
df=pd.read_table('banknotes.txt')
df.iloc[:,3:6].head()

,X4,X5,X6
0,9.0,9.7,141.0
1,8.1,9.5,141.7
2,8.7,9.6,142.2
3,7.5,10.4,142.0
4,10.4,7.7,141.8


In [59]:
X_train,X_test,y_train,y_test=train_test_split(df.iloc[:,:6],df.iloc[:,6],test_size=0.25,random_state=1)
lr1=LogisticRegression(solver='liblinear').fit(X_train.iloc[:,:3],y_train)
lr2=LogisticRegression(solver='liblinear').fit(X_train.iloc[:,3:6],y_train)

In [60]:
pred1=lr1.predict(X_test.iloc[:,:3])
pred2=lr2.predict(X_test.iloc[:,3:6])

In [61]:
def err_proportion(predicted,true):
    sample = list(zip(predicted, true))
    n = len(sample)
    
    return sum([0 if (x[0] == x[1]) else 1 for x in sample])/n

In [62]:
err1=err_proportion(pred1,y_test)
err2=err_proportion(pred2,y_test)
print(err1,err2)


0.2 0.02


In [65]:
p1=err1
p2=err2
n1=50
n2=50
P=float(p1*n1 + p2*n2) / (n1 + n2)
z=(p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))
z
p_val=2*(1-norm.cdf(z,0,1))
p_val

0.004022237272055307

=>доли ошибок не одинаковые

# 5
В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [70]:
def proportions_diff_confint_rel(sample1, sample2,y_true, alpha = 0.05):
    z = norm.ppf(1 - alpha / 2.)
    
    sample=list(zip(pred1,pred2,y_true))
    n=len(y_true)
    f=sum([1 if x[0]==x[2] and x[1]!=x[2] else 0 for x in sample])
    g=sum([1 if x[0]!=x[2] and x[1]==x[2] else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [73]:
round(proportions_diff_confint_rel(pred1,pred2,y_test)[1],4)

-0.0599

# 6
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов. 

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

$H_0\colon$ программа неэффективна, т.е. $\mu$=$\mu_0\colon$

$H_1\colon$ эффективна, т.е.  $\mu$>$\mu_0\colon$

Используем одновыборочный z-критерий


In [75]:
z=(541.4-525)/(100/10)
z

1.6399999999999977

In [78]:
p_value=1-norm.cdf(z,0,1)
round(p_value,4)

0.0505

=> нулевая гипотеза не отвергается

# 7
Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки. 

In [80]:
z=(541.5-525)/(100/10)
z

1.65

In [81]:
p_value=1-norm.cdf(z,0,1)
round(p_value,4)

0.0495

In [ ]:
=> нулевая гипотеза отвергается, программа эффективна